# Librerías 

In [ ]:
from pathlib import Path
import numpy
import os
import parmed.amber
from datetime import datetime

# Directorios y Variables


In [ ]:
#Dirección de MSMS, datos de entrada en .prmtop y .crd, dirección de salida
msms = '/home/rodrigo/codigo/msms.1'  #ubicación de MSMS
direccion_datos = '/home/rodrigo/Desktop/datos/'  #ubicación de moleculas a transformar
direccion_proteinas = '/home/rodrigo/Desktop/proteinas/'  #dirección a ubicar datos para pygbe

In [ ]:
#Variables que se pueden modificar

#Elegir si utilizar Radio 'Born' (1) o Radio 'Lennard-Jones VdW'  (2)
OPCION = 2 #Valores 1 o 2


#Datos de entrada para MSMS:
radius_increase=0
p=1.4 #Probe Radius para generar SES
d=3.0 #densidad, valores usuales 1.0 (>1000) y 3.0 para moléculas más pequeñas


#Parámetros para cambiar en .parm
NCRIT = 500
theta = 0.5


#Parámetros para cambiar en .config
epsilon = 1.15

# Códigos y Funciones

## Genarar Directorios y Mallas


In [ ]:
#Lista de Moleculas en la Carpeta.  ### 1.-
def lista_molec(direccion):
    lista_de_moleculas=[]
    for dirpath ,dirnames, filenames in os.walk(direccion):
        filenames
        for molecula in filenames:
            if '.crd' in molecula:
                molecula=molecula[:-4]
                lista_de_moleculas.append(molecula)

            elif '.pqr' in molecula:
                molecula=molecula[:-4]
                lista_de_moleculas.append(molecula)
    return lista_de_moleculas


#-----------------------------------------------------------------------------------------

#Crear Directorio por cada Molecula.  ### 2.-
def dir_molec(direccion_proteinas,molecule_name):
    return os.system('mkdir '+direccion_proteinas + molecule_name)


#-----------------------------------------------------------------------------------------


#Definir Posiciones, radios y los tipos de átomos para cada molécula.   ### 3.-

def pos_radi(molecule_name,OPCION,direccion_datos):
    mol_param = parmed.amber.AmberParm(direccion_datos + molecule_name + '.prmtop')
    N_atom = mol_param.ptr('NATOM')
    atom_type = mol_param.parm_data['ATOM_TYPE_INDEX']
    atom_radius = numpy.zeros(N_atom)
    atom_depth = numpy.zeros(N_atom)
    
    if OPCION == 1:
        atom_radius[:] = mol_param.parm_data['RADII'][:]

        if N_atom%2==0:
            mol_crd = numpy.loadtxt(direccion_datos+molecule_name+'.crd',skiprows=2)
            mol_crd.flatten()
            atom_pos = numpy.reshape(mol_crd,(N_atom,3))

        elif N_atom%2!=0:
            maximas_filas = (N_atom-1)//2
            mol_crd=numpy.zeros(N_atom*3)
            mol_crd_ini = numpy.loadtxt(direccion_datos+molecule_name+'.crd',skiprows=2,max_rows=maximas_filas)
            mol_crd_end = numpy.loadtxt(direccion_datos+molecule_name+'.crd',skiprows=2+maximas_filas)
            mol_crd[N_atom*3-3:] = mol_crd_end[:]
            mol_crd[:N_atom*3-3] = mol_crd_ini.flatten()
            atom_pos = numpy.reshape(mol_crd, (N_atom, 3))
            
    elif OPCION == 2:
        for i in range(N_atom):
            atom_radius[i] = mol_param.LJ_radius[atom_type[i]-1] 
    

        if N_atom%2==0:
            mol_crd = numpy.loadtxt(direccion_datos+molecule_name+'.crd',skiprows=2)
            mol_crd.flatten()
            atom_pos = numpy.reshape(mol_crd,(N_atom,3))

        elif N_atom%2!=0:
            maximas_filas = (N_atom-1)//2
            mol_crd = numpy.zeros(N_atom*3)
            mol_crd_ini = numpy.loadtxt(direccion_datos+molecule_name+'.crd',skiprows=2,max_rows=maximas_filas)
            mol_crd_end = numpy.loadtxt(direccion_datos+molecule_name+'.crd',skiprows=2+maximas_filas)
            mol_crd[N_atom*3-3:] = mol_crd_end[:]
            mol_crd[:N_atom*3-3] = mol_crd_ini.flatten()
            atom_pos = numpy.reshape(mol_crd, (N_atom, 3))
        
    return atom_pos, atom_radius, N_atom, atom_type
       

#-----------------------------------------------------------------------------------------

         
#Generar .xyzr y Malla con MSMS.
##Variables Pueden ser cambiadas en sección anterior.  ### 4.-

def convert_mesh(atom_pos, atom_radius, msms, molecule_name, direccion_proteinas, radius_increase, p, d):
    os.system('mkdir '+direccion_proteinas+'/'+molecule_name+'/geometry')
    xyzr_data = numpy.zeros((N_atom,4))
    xyzr_data[:,:3] = atom_pos[:,:]
    xyzr_data[:,-1] = atom_radius[:] + radius_increase
    
    
    #generate xyzr and call MSMS.
    dir_x_molec = direccion_proteinas+molecule_name  #Nombre de los directorios.
    dir_x_molec_name = dir_x_molec+'/'+molecule_name
    
    numpy.savetxt(dir_x_molec_name+'.xyzr',xyzr_data)
    cmd = msms+' -if '+dir_x_molec_name+'.xyzr -of '+dir_x_molec+'/geometry/'+\
    molecule_name + (' -d %1.1f -p %1.1f -no_header') %(d,p)
    return os.system(cmd)

## Generar .config, .param, y .pqr

In [ ]:
#Generar .config
def gen_config(molecule_name):
    g=open(direccion_proteinas+molecule_name+'/'+molecule_name+'.config','w+')
    config_text = \
    'FILE    geometry    dielectric_interface\n\
    --------------------------------\n\
    PARAM   LorY E?   Dielec  kappa   charges?    coulomb?  charge_file     Nparent parent  Nchild  children\n\
    FIELD   1    0      80    1e-12       0           0       NA              0       NA      1       0\n\
    FIELD   1    1      %1.2f   1e-12       1           1       MOLECULE        1       0       0       NA'\
    %(epsilon)
    config_text_mod1=config_text.replace('geometry','geometry/'+molecule_name)
    config_text_mod2=config_text_mod1.replace('MOLECULE',molecule_name+'.pqr')
    g.write(config_text_mod2)
    return g.close()

In [ ]:
#Generar .param
def gen_param(molecule_name):
    f=open(direccion_proteinas+molecule_name+'/'+molecule_name+'.param','w+')
    text_param = 'Precision   double\n\
K           4\n\
Nk          9\n\
K_fine      37\n\
thresold    0.5\n\
BSZ         128\n\
restart     100\n\
tolerance   1e-6\n\
max_iter    1000\n\
P           8\n\
eps         1e-12\n\
NCRIT       %1.0f\n\
theta       %1.2f\n\
GPU         0\n\
polar_eps   1e-2'%(NCRIT, theta)
    f.write(text_param)
    return f.close()

In [ ]:
#Generar .pqr
def gen_pqr(molecule_name):
    dir_name = direccion_datos+molecule_name
    mol = parmed.amber.AmberParm(direccion_datos + molecule_name + '.prmtop')
    N_atom = mol.ptr('NATOM')
    nombre = mol.parm_data['ATOM_NAME']
    tipo = mol.parm_data['ATOM_TYPE_INDEX']
    carga = mol.parm_data['CHARGE']
    radio = numpy.zeros(N_atom)
    
    if OPCION == 1:
        radio[:] = mol.parm_data['RADII'][:]    
        
        if N_atom%2==0:
            mol_crd = numpy.loadtxt(direccion_datos+molecule_name+'.crd',skiprows=2)
            mol_crd.flatten()
            pos = numpy.reshape(mol_crd,(N_atom,3))

        elif N_atom%2!=0:
            maximas_filas=(N_atom-1)//2
            mol_crd=numpy.zeros(N_atom*3)
            mol_crd_ini= numpy.loadtxt(direccion_datos+molecule_name+'.crd',skiprows=2,max_rows=maximas_filas)
            mol_crd_end = numpy.loadtxt(direccion_datos+molecule_name+'.crd',skiprows=2+maximas_filas)
            mol_crd[N_atom*3-3:]=mol_crd_end[:]
            mol_crd[:N_atom*3-3]=mol_crd_ini.flatten()
            pos = numpy.reshape(mol_crd, (N_atom, 3))
            
    if OPCION == 2:
        for i in range(N_atom):
            radio[i] = mol.LJ_radius[atom_type[i]-1] 
    
    
        if N_atom%2==0:
            mol_crd = numpy.loadtxt(direccion_datos+molecule_name+'.crd',skiprows=2)
            mol_crd.flatten()
            pos = numpy.reshape(mol_crd,(N_atom,3))

        elif N_atom%2!=0:
            maximas_filas=(N_atom-1)//2
            mol_crd=numpy.zeros(N_atom*3)
            mol_crd_ini= numpy.loadtxt(direccion_datos+molecule_name+'.crd',skiprows=2,max_rows=maximas_filas)
            mol_crd_end = numpy.loadtxt(direccion_datos+molecule_name+'.crd',skiprows=2+maximas_filas)
            mol_crd[N_atom*3-3:]=mol_crd_end[:]
            mol_crd[:N_atom*3-3]=mol_crd_ini.flatten()
            pos = numpy.reshape(mol_crd, (N_atom, 3))

            
            
            
    h=open(direccion_proteinas+molecule_name+'/'+molecule_name+'.pqr','w+')
    for i in range(N_atom):
        h.write('ATOM   %1.0f   %s   ALGO    %1.0f   %1.3f  %1.3f  %1.3f %1.4f %1.5f \n'%(i+1,nombre[i],tipo[i],pos[i][0],pos[i][1],pos[i][2],carga[i],radio[i])) 
    return h.close()


## Correr PyGBe y gen .txt con los resultados

In [ ]:
def call_pygbe(molecule):
    os.system('pygbe '+ direccion_proteinas+ '/' +molecule)

In [ ]:
def gen_results(direccion_proteinas,lista):
    g=open(direccion_proteinas+'resultados '+str(datetime.now())[:-7] +' .txt','w+')
    for name in lista:
        logs=[]
        for a,b, filenames in os.walk(direccion_proteinas+name+'/output'):
            for log in filenames:
                if '.log' in log:
                    logs.append(log)
        f=open(direccion_proteinas+name+'/output/'+logs[-1])
        E_solv=[]
        while(True):
            linea=f.readline()
            if 'E_solv = ' in linea:
                E_solv.append(linea)

            if not linea:
                break
        f.close()
        os.system('mv ' +direccion_proteinas+name+'/output/'+logs[-1] + ' ' + direccion_proteinas+name+'/output/'+logs[-1][:-4]+\
                  '.revisado')
        del logs
        if E_solv!=0:
            g.write(name+'   '+E_solv[-1])
            #g.write(' \n')  ###Activar en caso de que haya error con alguna molécula
        del E_solv


    return g.close()


# Correr Funciones

In [ ]:
## Generar lista, directorios y mallas(Solo una vez):
list_molecules = lista_molec(direccion_datos)
for molec_name in list_molecules:
    dir_molec(direccion_proteinas,molec_name)
    #print (molec_name)             ### En caso de Haber error con alguna Molec.
    atom_pos, atom_radius, N_atom, atom_type = pos_radi(molec_name,OPCION,direccion_datos)
    convert_mesh(atom_pos, atom_radius, msms, molec_name, direccion_proteinas, radius_increase, p, d)


In [ ]:
##Generar .param, .config, .pqr:
for molec_name in list_molecules:
    gen_config(molec_name)
    gen_param(molec_name)
    gen_pqr(molec_name)
    

In [ ]:
#PyGBe y data gen:
for molec_name in list_molecules:
    call_pygbe(molec_name)   

In [ ]:
gen_results(direccion_proteinas,list_molecules)